In [1]:
# Ensure reproducibility

import numpy as np

CUSTOM_SEED = 42
np.random.seed(CUSTOM_SEED)
import resource

In [2]:
import nltk
nltk.download('treebank')
nltk.download('maxent_treebank_pos_tagger')

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


[nltk_data] Downloading package treebank to /home/linu/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/linu/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
import random
from nltk.corpus import treebank
sentences = treebank.tagged_sents()
print('a random sentence: \n-> {}'.format(random.choice(sentences)))

a random sentence: 
-> [('Congress', 'NNP'), ('learned', 'VBD'), ('during', 'IN'), ('the', 'DT'), ('Reagan', 'NNP'), ('administration', 'NN'), ('that', 'IN'), ('it', 'PRP'), ('could', 'MD'), ('intimidate', 'VB'), ('the', 'DT'), ('executive', 'JJ'), ('branch', 'NN'), ('by', 'IN'), ('*-1', '-NONE-'), ('uttering', 'VBG'), ('again', 'RB'), ('and', 'CC'), ('again', 'RB'), ('the', 'DT'), ('same', 'JJ'), ('seven', 'CD'), ('words', 'NNS'), (':', ':'), ('``', '``'), ('Provided', 'VBN'), (',', ','), ('that', 'IN'), ('no', 'DT'), ('funds', 'NNS'), ('shall', 'MD'), ('be', 'VB'), ('spent', 'VBN'), ('*-48', '-NONE-'), ('...', ':'), ('.', '.'), ("''", "''")]


In [4]:
tags = set([tag for sentence in treebank.tagged_sents() for _, tag in sentence])
print('nb_tags: {}\ntags: {}'.format(len(tags), tags))

nb_tags: 46
tags: {':', 'PRP', 'WP$', 'RBS', 'VBN', 'PRP$', 'VBD', '``', 'VBP', 'NN', 'JJ', '-NONE-', 'IN', 'UH', 'NNPS', 'RB', 'POS', 'TO', '-LRB-', 'CD', "''", 'NNS', 'WRB', 'FW', 'JJR', '-RRB-', 'WP', '#', 'DT', 'MD', '$', 'EX', 'PDT', ',', 'VB', 'WDT', 'JJS', 'SYM', 'RP', 'VBZ', 'VBG', 'NNP', 'LS', '.', 'CC', 'RBR'}


In [5]:
train_test_cutoff = int(.80 * len(sentences)) 
training_sentences = sentences[:train_test_cutoff]
testing_sentences = sentences[train_test_cutoff:]
 
train_val_cutoff = int(.25 * len(training_sentences))
validation_sentences = training_sentences[:train_val_cutoff]
training_sentences = training_sentences[train_val_cutoff:]

In [6]:

def add_basic_features(sentence_terms, index):
    """ Compute some very basic word features.

        :param sentence_terms: [w1, w2, ...] 
        :type sentence_terms: list
        :param index: the index of the word 
        :type index: int
        :return: dict containing features
        :rtype: dict
    """
    term = sentence_terms[index]
    return {
        'nb_terms': len(sentence_terms),
        'term': term,
        'is_first': index == 0,
        'is_last': index == len(sentence_terms) - 1,
        'is_capitalized': term[0].upper() == term[0],
        'is_all_caps': term.upper() == term,
        'is_all_lower': term.lower() == term,
        'prefix-1': term[0],
        'prefix-2': term[:2],
        'prefix-3': term[:3],
        'suffix-1': term[-1],
        'suffix-2': term[-2:],
        'suffix-3': term[-3:],
        'prev_word': '' if index == 0 else sentence_terms[index - 1],
        'next_word': '' if index == len(sentence_terms) - 1 else sentence_terms[index + 1]
    }

In [7]:
def untag(tagged_sentence):
    """ 
    Remove the tag for each tagged term. 

    :param tagged_sentence: a POS tagged sentence
    :type tagged_sentence: list
    :return: a list of tags
    :rtype: list of strings
    """
    return [w for w, _ in tagged_sentence]

def transform_to_dataset(tagged_sentences):
    """
    Split tagged sentences to X and y datasets and append some basic features.

    :param tagged_sentences: a list of POS tagged sentences
    :param tagged_sentences: list of list of tuples (term_i, tag_i)
    :return: 
    """
    X, y = [], []

    for pos_tags in tagged_sentences:
        for index, (term, class_) in enumerate(pos_tags):
            # Add basic NLP features for each sentence term
            X.append(add_basic_features(untag(pos_tags), index))
            y.append(class_)
    return X, y

In [8]:

X_train, y_train = transform_to_dataset(training_sentences)
X_test, y_test = transform_to_dataset(testing_sentences)
X_val, y_val = transform_to_dataset(validation_sentences)

In [9]:
from sklearn.feature_extraction import DictVectorizer
 
dict_vectorizer = DictVectorizer(sparse=False)
dict_vectorizer.fit(X_train + X_test + X_val)

DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=False)

In [10]:
X_train = dict_vectorizer.transform(X_train)
X_test = dict_vectorizer.transform(X_test)
X_val = dict_vectorizer.transform(X_val)

In [11]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('X_train', 21623078704),
 ('X_test', 7090920496),
 ('X_val', 6910807792),
 ('y_train', 514568),
 ('y_test', 178024),
 ('y_val', 158200),
 ('tags', 2272),
 ('DictVectorizer', 1304),
 ('add_basic_features', 136),
 ('transform_to_dataset', 136),
 ('untag', 136),
 ('np', 80),
 ('dict_vectorizer', 56),
 ('sentences', 56),
 ('testing_sentences', 56),
 ('training_sentences', 56),
 ('treebank', 56),
 ('validation_sentences', 56),
 ('CUSTOM_SEED', 28),
 ('train_test_cutoff', 28),
 ('train_val_cutoff', 28)]

In [12]:
from sklearn.preprocessing import LabelEncoder
# Fit LabelEncoder with our list of classes
label_encoder = LabelEncoder()
label_encoder.fit(y_train + y_test + y_val)
# Encode class values as integers
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)
y_val = label_encoder.transform(y_val)

In [13]:
# Convert integers to dummy variables (one hot encoded)
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
y_val = np_utils.to_categorical(y_val)

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
def build_model(input_dim, hidden_neurons, output_dim):
    """
    Construct, compile and return a Keras model which will be used to fit/predict
    """
    model = Sequential([
        Dense(hidden_neurons, input_dim=input_dim),
        Activation('relu'),
        Dropout(0.2),
        Dense(hidden_neurons),
        Activation('relu'),
        Dropout(0.2),
        Dense(output_dim, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [15]:
from keras.wrappers.scikit_learn import KerasClassifier
model_params = {
    'build_fn': build_model,
    'input_dim': X_train.shape[1],
    'hidden_neurons': 512,
    'output_dim': y_train.shape[1],
    'epochs': 5,
    'batch_size': 256,
    'verbose': 1,
    'validation_data': (X_val, y_val),
    'shuffle': True
}
clf = KerasClassifier(**model_params)

In [ ]:
hist = clf.fit(X_train, y_train)

Train on 61107 samples, validate on 19530 samples
Epoch 1/5
60928/61107 [============================>.] - ETA: 6s - loss: 0.6938 - acc: 0.8293 